In [1]:
!apt-get install cuda-9.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libcuda-9.2-1' for regex 'cuda-9.2'
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [12]:
%%writefile cudabfs.cu
#include<iostream>
#include<cuda.h>
#include <stdio.h>
#define INF 1e9;

using namespace std;




__global__
void bfsUtil(int dptrVertices[], int dptrEdges[], int* dn,
    int* dm, int dlevelOrder[], int* dptrLevel, int* dptrVisited, int* dptrFlag)
{
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (dlevelOrder[id] == *dptrLevel)
    {
        int startingidx;
        int endingidx;
        if (id != *dn - 1)
        {
            startingidx = dptrVertices[id];
            endingidx = dptrVertices[id + 1] - 1;
        }
        else
        {
            startingidx = dptrVertices[id];
            endingidx = *dm - 1;
        }
        for (int i = startingidx; i <= endingidx; i++)
        {
            if (!dptrVisited[dptrEdges[i]])
            {
                dlevelOrder[dptrEdges[i]]=*dptrLevel + 1;
                dptrVisited[dptrEdges[i]]=1;
                atomicExch(dptrFlag, 1);
               
                
               
            }
        }
    }
}

void bfs(int vertices[], int edges[], int n, int m, int src)
{
    int* levelOrder = new int[n];
    int level = 0;
    int flag = 1;
    int* dv;
    int* de;
    int* dn;
    int* dm;
    int* visited = new int[n];

    for (int i = 0; i < n; i++)
    {
        visited[i] = 0;
        levelOrder[i] = INF;
    }
    levelOrder[src] = 0;
    visited[src] = 1;


  
    cudaMalloc(&dv, n * sizeof(int));
    cudaMalloc(&de, m * sizeof(int));
    cudaMalloc(&dn, sizeof(int));
    cudaMalloc(&dm, sizeof(int));

  
    cudaMemcpy(dv, vertices, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(de, edges, m * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dn, &n, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dm, &m, sizeof(int), cudaMemcpyHostToDevice);
    while (flag)
    {
        flag = 0;
        int* dlevelOrder;
        int* dl;
        int* di;
        int* dvisited;
        int* dflag;

    
        cudaMalloc(&dvisited, sizeof(int) * n);
        cudaMalloc(&dlevelOrder, sizeof(int) * n);
        cudaMalloc(&dl, sizeof(int));
        cudaMalloc(&di, sizeof(int));
        cudaMalloc(&dflag, sizeof(int));

      
        cudaMemcpy(dvisited, visited, sizeof(int) * n, cudaMemcpyHostToDevice);
        cudaMemcpy(dlevelOrder, levelOrder, n * sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(dl, &level, sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(dflag, &flag, sizeof(int), cudaMemcpyHostToDevice);


        
        bfsUtil << <1, n >> > (dv, de, dn, dm, dlevelOrder, dl, dvisited, dflag);

      
        cudaMemcpy(levelOrder, dlevelOrder, n * sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(&level, dl, sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(visited, dvisited, n * sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(&flag, dflag, sizeof(int), cudaMemcpyDeviceToHost);



        cudaFree(dlevelOrder);
        cudaFree(dl);
        level++;

    }


    for (int i = 0; i < n; i++)
    {
    
        std::cout << "node " << i << " has level " << levelOrder[i] << " ";
        std::cout << std::endl;
    }

    cudaFree(dv);
    cudaFree(de);
    cudaFree(dn);
    cudaFree(dm);
}
int main(void)
{
    int edges[] = { 1,2,3,0,0,4,5,0,2,2 };
    int vertices[] = { 0,3,4,7,8,9 };
    int source = 0;
    int n = sizeof(vertices) / sizeof(vertices[0]);
    int m = sizeof(edges) / sizeof(edges[0]);

    bfs(vertices, edges, n, m, source);
    return 0;
}



Overwriting cudabfs.cu


In [13]:
!nvcc cudabfs.cu
!./a.out

node 0 has level 0 
node 1 has level 1 
node 2 has level 1 
node 3 has level 1 
node 4 has level 2 
node 5 has level 2 
